In [48]:
import pysam
import pandas as pd 
from Bio import pairwise2 as pw2
import sys
import glob
from collections import defaultdict

In [49]:
def percent_match(seq1,seq2):
    global_align = pw2.align.globalxx(seq1,seq2)
    seq_length = min(len(seq1),len(seq2))
    matches = global_align[0][2]
    percent_match = (matches/seq_length) * 100
    return percent_match

In [50]:
"""
对照	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220815/H_0727PBMC2_K_T3Nlib/
测试1	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220815/H_0727PBMC2_Mmix_T3Nlib/
测试2	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220817/H_0727PBMC2_M_T3Nlib/
	
对照	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220811_2/M_0727PBMC2_NK_T3Nlib/
测试1	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220811_2/M_0727PBMC2_NMc2_T3Nlib/
	
对照	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220811_2/M_0727PBMC1_NK_T3Nlib/
测试1	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220811_2/M_0727PBMC1_NMc2_T3Nlib/
	
对照	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220721/M_0705Pz1_NK_T3Nlib/
测试1	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220721/M_0705Pz1_NMC2_T3Nlib/
	
对照	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220721/M_0705Pz2_NK_T3Nlib/
测试1	/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220721/M_0705Pz2_NMC2_T3Nlib/

"""

'\n对照\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220815/H_0727PBMC2_K_T3Nlib/\n测试1\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220815/H_0727PBMC2_Mmix_T3Nlib/\n测试2\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220817/H_0727PBMC2_M_T3Nlib/\n\t\n对照\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220811_2/M_0727PBMC2_NK_T3Nlib/\n测试1\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220811_2/M_0727PBMC2_NMc2_T3Nlib/\n\t\n对照\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220811_2/M_0727PBMC1_NK_T3Nlib/\n测试1\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220811_2/M_0727PBMC1_NMc2_T3Nlib/\n\t\n对照\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220721/M_0705Pz1_NK_T3Nlib/\n测试1\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220721/M_0705Pz1_NMC2_T3Nlib/\n\t\n对照\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220721/M_0705Pz2_NK_T3Nlib/\n测试1\t/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220721/M_0705Pz2_NMC2_T3Nlib/\n\n'

In [51]:
test = '/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220817/H_0727PBMC2_M_T3Nlib/06.summarize/productive/productive_match_contig_annotations.csv'
test_fasta = '/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220817/H_0727PBMC2_M_T3Nlib/06.summarize/productive/productive_match_contig.fasta'
comp = '/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220815/H_0727PBMC2_K_T3Nlib/06.summarize/productive/productive_match_contig_annotations.csv'
comp_fasta = '/SGRNJ06/randd/PROJECT/RD20073101_VDJ/LZL_2022/20220815/H_0727PBMC2_K_T3Nlib/06.summarize/productive/productive_match_contig.fasta'

In [52]:
test_csv = pd.read_csv(test)
comp_csv = pd.read_csv(comp)

In [53]:
test_A = test_csv[test_csv['chain']=='TRA']
test_B = test_csv[test_csv['chain']=='TRB']
comp_A = comp_csv[comp_csv['chain']=='TRA']
comp_B = comp_csv[comp_csv['chain']=='TRB']
intersec_A = set(test_A.barcode).intersection(comp_A.barcode)
intersec_B = set(test_B.barcode).intersection(comp_B.barcode)

In [54]:
test_A = test_A[test_A.barcode.isin(intersec_A)]
comp_A = comp_A[comp_A.barcode.isin(intersec_A)]
test_B = test_B[test_B.barcode.isin(intersec_B)]
comp_B = comp_B[comp_B.barcode.isin(intersec_B)]

In [55]:
test_A.sort_values(by=['barcode','umis'], ascending=[True,False],inplace=True)
comp_A.sort_values(by=['barcode','umis'], ascending=[True,False],inplace=True)
test_B.sort_values(by=['barcode','umis'], ascending=[True,False],inplace=True)
comp_B.sort_values(by=['barcode','umis'], ascending=[True,False],inplace=True)

In [56]:
test_A.drop_duplicates('barcode',inplace=True)
comp_A.drop_duplicates('barcode',inplace=True)
test_B.drop_duplicates('barcode',inplace=True)
comp_B.drop_duplicates('barcode',inplace=True)

In [57]:
test_A

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
2,AAACATCGACATTGGCCGCATACA,True,AAACATCGACATTGGCCGCATACA_contig_1,True,618,TRA,TRAV3,None,TRAJ34,TRAC,True,True,CAVRETYNTDKLIF,TGTGCTGTGAGAGAAACTTATAACACCGACAAGCTCATCTTT,21220,5,clonotype3,clonotype3_consensus_1
5,AAACATCGACTATGCAACAAGCTA,True,AAACATCGACTATGCAACAAGCTA_contig_2,True,667,TRA,TRAV39,None,TRAJ27,TRAC,True,True,CAVGYGGKSTF,TGTGCCGTGGGCTATGGAGGCAAATCAACCTTT,14947,3,clonotype269,clonotype269_consensus_2
7,AAACATCGAGTACAAGGACTAGTA,True,AAACATCGAGTACAAGGACTAGTA_contig_2,True,532,TRA,TRAV29/DV5,None,TRAJ54,TRAC,True,True,CAASAMGAQKLVF,TGTGCAGCAAGCGCGATGGGAGCCCAGAAGCTGGTATTT,4963,1,clonotype1,clonotype1_consensus_1
11,AAACATCGCATCAAGTAGTACAAG,True,AAACATCGCATCAAGTAGTACAAG_contig_2,True,531,TRA,TRAV12-2,None,TRAJ48,TRAC,True,True,CAVEENFGNEKLTF,TGTGCCGTGGAGGAGAACTTTGGAAATGAGAAATTAACCTTT,11701,4,clonotype2,clonotype2_consensus_1
15,AAACATCGCGCTGATCAACGTGAT,True,AAACATCGCGCTGATCAACGTGAT_contig_2,True,446,TRA,TRAV12-3,None,TRAJ26,TRAC,True,True,CAMSADYGQNFVF,TGTGCAATGAGCGCGGACTATGGTCAGAATTTTGTCTTT,28,4,clonotype258,clonotype258_consensus_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,TTCACGCAAGATGTACCCTAATCC,True,TTCACGCAAGATGTACCCTAATCC_contig_2,True,726,TRA,TRAV12-1,None,TRAJ26,TRAC,True,True,CVVNDNYGQNFVF,TGTGTGGTGAACGATAACTATGGTCAGAATTTTGTCTTT,56437,7,clonotype14,clonotype14_consensus_1
2563,TTCACGCAAGCAGGAAACATTGGC,True,TTCACGCAAGCAGGAAACATTGGC_contig_1,True,672,TRA,TRAV12-1,None,TRAJ23,TRAC,True,True,CVVGVGKLIF,TGTGTGGTGGGGGTAGGAAAGCTTATCTTC,8002,1,clonotype602,clonotype602_consensus_2
2566,TTCACGCAATTGAGGAAAGGACAC,True,TTCACGCAATTGAGGAAAGGACAC_contig_2,True,740,TRA,TRAV12-1,None,TRAJ15,TRAC,True,True,CVVNLNQAGTALIF,TGTGTGGTGAACCTCAACCAGGCAGGAACTGCTCTGATCTTT,25163,4,clonotype417,clonotype417_consensus_1
2573,TTCACGCAGTACGCAAAGTGGTCA,True,TTCACGCAGTACGCAAAGTGGTCA_contig_1,True,618,TRA,TRAV3,None,TRAJ34,TRAC,True,True,CAVRETYNTDKLIF,TGTGCTGTGAGAGAAACTTATAACACCGACAAGCTCATCTTT,12110,3,clonotype3,clonotype3_consensus_1


In [58]:
test_A_contig_set = set(test_A.contig_id)
with pysam.FastxFile(test_fasta) as f:
    for read in f:
        if read.name in test_A_contig_set:
            test_A.loc[test_A.contig_id==read.name, 'seq'] = read.sequence

In [59]:
test_B_contig_set = set(test_B.contig_id)
with pysam.FastxFile(test_fasta) as f:
    for read in f:
        if read.name in test_B_contig_set:
            test_B.loc[test_B.contig_id==read.name, 'seq'] = read.sequence

In [60]:
comp_A_contig_set = set(comp_A.contig_id)
with pysam.FastxFile(comp_fasta) as f:
    for read in f:
        if read.name in comp_A_contig_set:
            comp_A.loc[comp_A.contig_id==read.name, 'seq'] = read.sequence

In [61]:
comp_B_contig_set = set(comp_B.contig_id)
with pysam.FastxFile(comp_fasta) as f:
    for read in f:
        if read.name in comp_B_contig_set:
            comp_B.loc[comp_B.contig_id==read.name, 'seq'] = read.sequence

In [62]:
test_A

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id,seq
2,AAACATCGACATTGGCCGCATACA,True,AAACATCGACATTGGCCGCATACA_contig_1,True,618,TRA,TRAV3,None,TRAJ34,TRAC,True,True,CAVRETYNTDKLIF,TGTGCTGTGAGAGAAACTTATAACACCGACAAGCTCATCTTT,21220,5,clonotype3,clonotype3_consensus_1,CCGGTGTCGGGAAGCACCAGTGCCCTGAGGAAGGGCCATTTCCAAA...
5,AAACATCGACTATGCAACAAGCTA,True,AAACATCGACTATGCAACAAGCTA_contig_2,True,667,TRA,TRAV39,None,TRAJ27,TRAC,True,True,CAVGYGGKSTF,TGTGCCGTGGGCTATGGAGGCAAATCAACCTTT,14947,3,clonotype269,clonotype269_consensus_2,TACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCATGGAG...
7,AAACATCGAGTACAAGGACTAGTA,True,AAACATCGAGTACAAGGACTAGTA_contig_2,True,532,TRA,TRAV29/DV5,None,TRAJ54,TRAC,True,True,CAASAMGAQKLVF,TGTGCAGCAAGCGCGATGGGAGCCCAGAAGCTGGTATTT,4963,1,clonotype1,clonotype1_consensus_1,AAAGCACAGGAGGTTGAAGTCAGATTTGCAGCTTTCTAGGCAGGAG...
11,AAACATCGCATCAAGTAGTACAAG,True,AAACATCGCATCAAGTAGTACAAG_contig_2,True,531,TRA,TRAV12-2,None,TRAJ48,TRAC,True,True,CAVEENFGNEKLTF,TGTGCCGTGGAGGAGAACTTTGGAAATGAGAAATTAACCTTT,11701,4,clonotype2,clonotype2_consensus_1,ATGATTTTTAAAGTAGAAATATCCATTCCAGGTGCATTTTTTAAGG...
15,AAACATCGCGCTGATCAACGTGAT,True,AAACATCGCGCTGATCAACGTGAT_contig_2,True,446,TRA,TRAV12-3,None,TRAJ26,TRAC,True,True,CAMSADYGQNFVF,TGTGCAATGAGCGCGGACTATGGTCAGAATTTTGTCTTT,28,4,clonotype258,clonotype258_consensus_3,AAGAATGATGAAATCCTTGAGAGTTTTACTGGTGATCCTGTGGCTT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,TTCACGCAAGATGTACCCTAATCC,True,TTCACGCAAGATGTACCCTAATCC_contig_2,True,726,TRA,TRAV12-1,None,TRAJ26,TRAC,True,True,CVVNDNYGQNFVF,TGTGTGGTGAACGATAACTATGGTCAGAATTTTGTCTTT,56437,7,clonotype14,clonotype14_consensus_1,GAGATCTACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGTT...
2563,TTCACGCAAGCAGGAAACATTGGC,True,TTCACGCAAGCAGGAAACATTGGC_contig_1,True,672,TRA,TRAV12-1,None,TRAJ23,TRAC,True,True,CVVGVGKLIF,TGTGTGGTGGGGGTAGGAAAGCTTATCTTC,8002,1,clonotype602,clonotype602_consensus_2,ACCACCGAGATCTACACTCGTCGGCAGCGTCAGATGTGTATAAGAG...
2566,TTCACGCAATTGAGGAAAGGACAC,True,TTCACGCAATTGAGGAAAGGACAC_contig_2,True,740,TRA,TRAV12-1,None,TRAJ15,TRAC,True,True,CVVNLNQAGTALIF,TGTGTGGTGAACCTCAACCAGGCAGGAACTGCTCTGATCTTT,25163,4,clonotype417,clonotype417_consensus_1,CCTCCGAGATCTACACTCGTCGGCAGCGTCAGATGTGTATAAGAGA...
2573,TTCACGCAGTACGCAAAGTGGTCA,True,TTCACGCAGTACGCAAAGTGGTCA_contig_1,True,618,TRA,TRAV3,None,TRAJ34,TRAC,True,True,CAVRETYNTDKLIF,TGTGCTGTGAGAGAAACTTATAACACCGACAAGCTCATCTTT,12110,3,clonotype3,clonotype3_consensus_1,GGGGTGTCGGGAAGCACCAGTGCCCTGAGGAAGGGCCATTTCCAAA...


In [63]:
comp_A

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id,seq
1,AAACATCGACATTGGCCGCATACA,True,AAACATCGACATTGGCCGCATACA_contig_1,True,618,TRA,TRAV3,None,TRAJ34,TRAC,True,True,CAVRETYNTDKLIF,TGTGCTGTGAGAGAAACTTATAACACCGACAAGCTCATCTTT,19366,5,clonotype40,clonotype40_consensus_2,CCGGTGTCGGGAAGCACCAGTGCCCTGAGGAAGGGCCATTTCCAAA...
4,AAACATCGACTATGCAACAAGCTA,True,AAACATCGACTATGCAACAAGCTA_contig_1,True,433,TRA,TRAV39,None,TRAJ27,TRAC,True,True,CAVGYGGKSTF,TGTGCCGTGGGCTATGGAGGCAAATCAACCTTT,13960,4,clonotype293,clonotype293_consensus_2,AAAAAAAAAATGAAGAAGCTACTAGCAATGATTCTGTGGCTTCAAC...
7,AAACATCGAGTACAAGGACTAGTA,True,AAACATCGAGTACAAGGACTAGTA_contig_2,True,532,TRA,TRAV29/DV5,None,TRAJ54,TRAC,True,True,CAASAMGAQKLVF,TGTGCAGCAAGCGCGATGGGAGCCCAGAAGCTGGTATTT,5578,1,clonotype1,clonotype1_consensus_1,AAAGCACAGGAGGTTGAAGTCAGATTTGCAGCTTTCTAGGCAGGAG...
11,AAACATCGCATCAAGTAGTACAAG,True,AAACATCGCATCAAGTAGTACAAG_contig_2,True,531,TRA,TRAV12-2,None,TRAJ48,TRAC,True,True,CAVEENFGNEKLTF,TGTGCCGTGGAGGAGAACTTTGGAAATGAGAAATTAACCTTT,10925,4,clonotype2,clonotype2_consensus_1,ATGATTTTTAAAGTAGAAATATCCATTCCAGGTGCATTTTTTAAGG...
13,AAACATCGCGCTGATCAACGTGAT,True,AAACATCGCGCTGATCAACGTGAT_contig_2,True,489,TRA,TRAV4,None,TRAJ9,TRAC,True,True,CLVGLLNTGGFKTIF,TGCCTCGTGGGTCTCTTAAATACTGGAGGCTTCAAAACTATCTTT,93,1,clonotype626,clonotype626_consensus_2,ATTGCAGCTCAGACACAGCAAAAGAGCCTAGAACCTGGGTCCTAGT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,TTCACGCAAGATGTACCCTAATCC,True,TTCACGCAAGATGTACCCTAATCC_contig_2,True,774,TRA,TRAV12-1,None,TRAJ26,TRAC,True,True,CVVNDNYGQNFVF,TGTGTGGTGAACGATAACTATGGTCAGAATTTTGTCTTT,57130,6,clonotype15,clonotype15_consensus_1,CCCCCCCCCCCCCCCCCCTTTTTTTATGAGCCGGCGACCACCGAGA...
2273,TTCACGCAAGCAGGAAACATTGGC,True,TTCACGCAAGCAGGAAACATTGGC_contig_2,True,707,TRA,TRAV12-1,None,TRAJ23,TRAC,True,True,CVVGVGKLIF,TGTGTGGTGGGGGTAGGAAAGCTTATCTTC,8458,2,clonotype552,clonotype552_consensus_2,CCCCCCCCCCCCCCCCCCTTTTTACTGCTCCGGCGACCACCGAGAT...
2274,TTCACGCAATTGAGGAAAGGACAC,True,TTCACGCAATTGAGGAAAGGACAC_contig_1,True,776,TRA,TRAV12-1,None,TRAJ15,TRAC,True,True,CVVNLNQAGTALIF,TGTGTGGTGAACCTCAACCAGGCAGGAACTGCTCTGATCTTT,29943,6,clonotype396,clonotype396_consensus_1,CCCCCCCCCCCCTCCCTGTTTTTTATTATACGGAGACCACCGAGAT...
2277,TTCACGCAGTACGCAAAGTGGTCA,True,TTCACGCAGTACGCAAAGTGGTCA_contig_2,True,618,TRA,TRAV3,None,TRAJ34,TRAC,True,True,CAVRETYNTDKLIF,TGTGCTGTGAGAGAAACTTATAACACCGACAAGCTCATCTTT,14461,4,clonotype3,clonotype3_consensus_1,GGGGTGTCGGGAAGCACCAGTGCCCTGAGGAAGGGCCATTTCCAAA...


In [64]:
test_A_seq_list, comp_A_seq_list = test_A.seq.tolist(), comp_A.seq.tolist()
match_A_list = [percent_match(x,y) for x,y in zip(test_A_seq_list, comp_A_seq_list)]
test_B_seq_list, comp_B_seq_list = test_B.seq.tolist(), comp_B.seq.tolist()
match_B_list = [percent_match(x,y) for x,y in zip(test_B_seq_list, comp_B_seq_list)]

In [ ]:
match_A_list

In [ ]:
match_B_list

In [67]:
test_A['Similarity'] = match_A_list
comp_A['Similarity'] = match_A_list

In [68]:
test_B['Similarity'] = match_B_list
comp_B['Similarity'] = match_B_list

In [70]:
test_A.to_csv('/SGRNJ06/randd/USER/cjj/celedev/Multiplex_KAPA_seq/20220909/H_0727PBMC2_M_T3Nlib/test_TRA.csv', sep=',', index=False)
comp_A.to_csv('/SGRNJ06/randd/USER/cjj/celedev/Multiplex_KAPA_seq/20220909/H_0727PBMC2_M_T3Nlib/cont_TRA.csv', sep=',', index=False)
test_B.to_csv('/SGRNJ06/randd/USER/cjj/celedev/Multiplex_KAPA_seq/20220909/H_0727PBMC2_M_T3Nlib/test_TRB.csv', sep=',', index=False)
comp_B.to_csv('/SGRNJ06/randd/USER/cjj/celedev/Multiplex_KAPA_seq/20220909/H_0727PBMC2_M_T3Nlib/cont_TRB.csv', sep=',', index=False)

In [ ]:
test_A

In [ ]:
comp_A

In [69]:
test_B

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id,seq,Similarity
0,AAACATCGAACGCTTAACCACTGT,True,AAACATCGAACGCTTAACCACTGT_contig_1,True,669,TRB,TRBV30,None,TRBJ2-7,TRBC2,True,True,CAWSPMADSYEQYF,TGTGCCTGGAGTCCTATGGCGGATTCCTACGAGCAGTACTTC,9073,3,clonotype2,clonotype2_consensus_2,CCGAGATCTACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAG...,100.000000
3,AAACATCGACATTGGCCGCATACA,True,AAACATCGACATTGGCCGCATACA_contig_2,True,469,TRB,TRBV28,None,TRBJ1-2,TRBC1,True,True,CASTPDVGTYGYTF,TGTGCCAGCACCCCGGACGTGGGGACCTATGGCTACACCTTC,40366,5,clonotype3,clonotype3_consensus_2,ATTCTTTCTTCAAAGCAGCCATGGGAATCAGGCTCCTCTGTCGTGT...,100.000000
4,AAACATCGACTATGCAACAAGCTA,True,AAACATCGACTATGCAACAAGCTA_contig_1,True,721,TRB,TRBV5-1,None,TRBJ2-5,TRBC2,True,True,CASSLTVTGEETQYF,TGCGCCAGCAGCTTGACGGTGACGGGAGAAGAGACCCAGTACTTC,56850,13,clonotype269,clonotype269_consensus_1,CCCCCCCCCCCCCCCCCCTCTTTTCATGATACGCCGACCACCCAGA...,98.333333
6,AAACATCGAGTACAAGGACTAGTA,True,AAACATCGAGTACAAGGACTAGTA_contig_1,True,501,TRB,TRBV6-6,None,TRBJ1-6,TRBC1,True,True,CASSNGISSYNSPLHF,TGTGCCAGCAGCAACGGCATAAGCTCCTATAATTCACCCCTCCACTTT,42796,10,clonotype1,clonotype1_consensus_2,ATCAATGCACAGATACAGAAGACCCCTCCGTCCTGGAGCACCTGCC...,100.000000
10,AAACATCGCATCAAGTAGTACAAG,True,AAACATCGCATCAAGTAGTACAAG_contig_1,True,537,TRB,TRBV30,None,TRBJ2-7,TRBC2,True,True,CAWSPMADSYEQYF,TGTGCCTGGAGTCCTATGGCGGATTCCTACGAGCAGTACTTC,15610,5,clonotype2,clonotype2_consensus_2,AGATCTCCCAGAGGGAGCAGCCTGACCACATCACTGGCCCAGAAGA...,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2561,TTCACGCAAGATGTACCCTAATCC,True,TTCACGCAAGATGTACCCTAATCC_contig_1,True,728,TRB,TRBV7-9,None,TRBJ2-3,TRBC2,True,True,CASSLFGLAGGDTQYF,TGTGCCAGCAGCTTATTTGGACTAGCGGGAGGGGATACGCAGTATTTT,14715,14,clonotype14,clonotype14_consensus_2,CCCCCCCTCCCCCCTTTCTTTTTTATGATTCGGCGACAACCGAGTT...,100.000000
2564,TTCACGCAAGCAGGAAACATTGGC,True,TTCACGCAAGCAGGAAACATTGGC_contig_2,True,476,TRB,TRBV15,None,TRBJ1-6,TRBC1,True,True,CATGIGQVNSPLHF,TGTGCCACCGGAATCGGACAAGTTAATTCACCCCTCCACTTT,45989,6,clonotype602,clonotype602_consensus_1,ATTCCTGTATGGGGTGGTATTCCTGCCATGGGTCCTGGGCTTCTCC...,100.000000
2565,TTCACGCAATTGAGGAAAGGACAC,True,TTCACGCAATTGAGGAAAGGACAC_contig_1,True,699,TRB,TRBV5-5,None,TRBJ1-3,TRBC1,True,True,CASSSGQVISGNTIYF,TGTGCCAGCAGTTCGGGACAGGTCATCTCTGGAAACACCATATATTTT,16436,5,clonotype417,clonotype417_consensus_2,CCTCCGAGATCTACACTCGTCGGCAGCGTCAGATGTGTATAAGAGA...,99.570815
2574,TTCACGCAGTACGCAAAGTGGTCA,True,TTCACGCAGTACGCAAAGTGGTCA_contig_2,True,469,TRB,TRBV28,None,TRBJ1-2,TRBC1,True,True,CASTPDVGTYGYTF,TGTGCCAGCACCCCGGACGTGGGGACCTATGGCTACACCTTC,45513,6,clonotype3,clonotype3_consensus_2,ATTCTTTCTTCAAAGCAGCCATGGGAATCAGGCTCCTCTGTCGTGT...,100.000000
